In [1]:
from scripts.data_util.sqlConnector import online_sql_connector , update_sql_since
connector = online_sql_connector()
update_sql_since('SellSideFactors' , connector = connector , trace = 0)
connector.close_all()

In [1]:
from scripts.data_util.sqlConnector import online_sql_connector ,  update_sql_dates
connector = online_sql_connector()
update_sql_dates('SellSideFactors' , connector = connector , dates=[20050104])
connector.close_all()


haitong/hf_factors


haitong/dl_factors


dongfang/hfq_chars
Thu Jan  4 07:32:52 2024 : dongfang:hfq_chars:20050104                    

dongfang/l2_chars


dongfang/ms_chars
Thu Jan  4 07:35:37 2024 : dongfang:ms_chars:20050104                    

dongfang/order_flow


dongfang/gp


dongfang/tra


dongfang/scores_v0


kaiyuan/positive


kaiyuan/negative


├── dongfang <HDF5 group "/dongfang" (5 members)>
│   ├── hfq_chars <HDF5 group "/dongfang/hfq_chars" (1 members)>
│   │   └── 20240102 <HDF5 group "/dongfang/hfq_chars/20240102" (3 members)> (feature, secid, values)
│   ├── l2_chars <HDF5 group "/dongfang/l2_chars" (1 members)>
│   │   └── 20240102 <HDF5 group "/dongfang/l2_chars/20240102" (3 members)> (feature, secid, values)
│   ├── ms_chars <HDF5 group "/dongfang/ms_chars" (1 members)>
│   │   └── 20240102 <HDF5 group "/dongfang/ms_chars/20240102" (3 members)> (feature, secid, values)
│   ├── order_flow <HDF5 group "/dongfang/order_flow" (1 members)>
│   │   └── 20240102 <HDF5 g

In [ ]:
#from scripts.data_util.DataTank import *
#from scripts.data_util.DataUpdater import *
#repack_DB_bygroup('data/DB_data/DB_3rdPartySQL.h5' , 'SellSideFactors' , True)